In [ ]:
!nvidia-smi

Thu Nov 14 12:04:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Downloading Necessary Libraries

In [1]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 21.0 MB/s eta 0:00:00
  

# Hugging face Logins

In [3]:
import os
from google.colab import userdata
os.environ["HF_Token"] = userdata.get('HF_Token')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Downloading Pre-trained Fine-tuned model(1 epoch) from Hugging Face Repo

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging
from transformers import logging as transformers_logging
import warnings

logging.basicConfig(level=logging.WARNING)
transformers_logging.set_verbosity_warning()
warnings.filterwarnings("ignore", category=UserWarning)

# Load the model and tokenizer
model_id = "DiatWork/GPT-Neox-MentalHealth-Finetune"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
model.config.use_cache = False  # Disable cache

tokenizer_config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/33.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/315M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

# Data Loading and Transformation

In [ ]:
from datasets import load_dataset, DatasetDict
#split the dataset
dataset = load_dataset("ShenLab/MentalChat16K")

train_test_split= dataset['train'].train_test_split(test_size=0.2)
train_dataset= train_test_split['train']
test_dataset= train_test_split['test']

train_valid_split= train_dataset.train_test_split(test_size=0.1)
train_dataset = train_valid_split["train"]
validation_dataset = train_valid_split["test"]

# Combine into a DatasetDict
split_dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

print(split_dataset)

dataset= split_dataset

README.md:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Interview_Data_6K.csv:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Synthetic_Data_10K.csv:   0%|          | 0.00/32.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16084 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 11580
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1287
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3217
    })
})


In [ ]:
#function to transform prompt
def transform_example(example):

    instruction = example['instruction'].strip()
    input_text = example['input'].strip() if example['input'] else ""
    output_text = example['output'].strip()


    if input_text:
        formatted_text = f"<s>[INST] {instruction} {input_text} [/INST] {output_text} </s>" #for cases when ther's no INST
    else:
        formatted_text = f"<s>[INST] {instruction} [/INST] {output_text} </s>"

    return {'text': formatted_text}

# transformer function call
transformed_dataset = DatasetDict({
    split: data.map(transform_example, remove_columns=data.column_names)
    for split, data in dataset.items()
})

print(transformed_dataset['train'][0]['text'])

Map:   0%|          | 0/11580 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Map:   0%|          | 0/3217 [00:00<?, ? examples/s]

<s>[INST] You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. 
The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. I've been feeling overwhelmed with caring for my father, who's in hospice care. The hospice nurses come three times a week to help with his showering needs, but I still feel responsible for his cleanliness and have to take care of him when they're not around. I'm worried about what happens if he has an accident and I have to clean him up. I can't bear the thought of it, and it keeps me up at night. [/INST] I hear your concerns, Mr. [Name]. It's understandable that you feel overwhelmed and anxious about caring for your father during this difficult time. Your feelings are valid, and it's essential to acknowledge the emotional and physical toll that caregiving can take. It's natural to feel a sense of responsibility towards your father, but it's also cruci

# Defining Q-LoRa Parameters

In [ ]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 10

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=['v_proj', 'up_proj', 'o_proj', 'q_proj', 'down_proj', 'gate_proj', 'k_proj'],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

#  Re-Training model on 10 epochs

In [ ]:
import transformers

from trl import SFTTrainer
from transformers import TrainingArguments

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset["train"],         # Specify only the train split
    eval_dataset=transformed_dataset["validation"],     # Specify validation split
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)


Map:   0%|          | 0/11580 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Step,Training Loss
25,1.321700
50,1.218000
75,1.318600
100,1.202800
125,1.272900
150,1.191200
175,1.306500
200,1.161000
225,1.269000
250,1.172900


Step,Training Loss
25,1.321700
50,1.218000
75,1.318600
100,1.202800
125,1.272900
150,1.191200
175,1.306500
200,1.161000
225,1.269000
250,1.172900


TrainOutput(global_step=28950, training_loss=0.923790482239402, metrics={'train_runtime': 18027.7392, 'train_samples_per_second': 6.423, 'train_steps_per_second': 1.606, 'total_flos': 9.74141441722368e+17, 'train_loss': 0.923790482239402, 'epoch': 10.0})

# Saving Model in Hugging Face Repository

In [ ]:
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
new_model = "GPT-Neox-MentalHealth-Finetune-10epoch"

trainer.model.save_pretrained(new_model)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


# Push the model and tokenizer to the Hugging Face Repo
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/315M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DiatWork/GPT-Neox-MentalHealth-Finetune-10epoch/commit/3de4c2c7a1fef3a9e4b98e60b538b0b0fda54368', commit_message='Upload tokenizer', commit_description='', oid='3de4c2c7a1fef3a9e4b98e60b538b0b0fda54368', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DiatWork/GPT-Neox-MentalHealth-Finetune-10epoch', endpoint='https://huggingface.co', repo_type='model', repo_id='DiatWork/GPT-Neox-MentalHealth-Finetune-10epoch'), pr_revision=None, pr_num=None)

# Load from Hugging Face Repo

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging
from transformers import logging as transformers_logging
import warnings

logging.basicConfig(level=logging.WARNING)
transformers_logging.set_verbosity_warning()
warnings.filterwarnings("ignore", category=UserWarning)

# Load the model and tokenizer
model_id = "DiatWork/GPT-Neox-MentalHealth-Finetune-10epoch"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
model.config.use_cache = False  # Disable cache

tokenizer_config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/33.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/315M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

# Inference

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
import torch
import textwrap

# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health assistant. I'm very stressed because I have a Job Interview, though I have practiced for it but this is my first interview so I'm anxious. How to deal with this stress? [/INST] "

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=400)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Question:
 <s>[INST] You are a helpful mental health assistant. I'm very stressed because I
have a Job Interview, though I have practiced for it but this is my first
interview so I'm anxious. How to deal with this stress?

Model Answer:
 It's completely normal to feel stressed during Job Interviews. The anticipation
and pressure can definitely take a toll on your mental well-being. Here are some
suggestions to help you cope with this stress:  1. Take it one day at a time:
Instead of focusing on the interview itself, try to take a step back and focus
on your overall well-being. This could involve practicing relaxation techniques
such as deep breathing exercises or meditation.  2. Prioritize self-care: Make
sure to take care of yourself physically and mentally. Engage in activities that
bring you joy and help you relax, whether it's reading a book, going for a walk,
or spending time with loved ones.  3. Set realistic expectations: It's important
to set realistic expectations for yourself

In [ ]:
# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health assistant. I'm very stressed because I have a group project in last semester of my college and I want to get first honours, I'm scared. Please help me cope with stress. [/INST]"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=200)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question:
 <s>[INST] You are a helpful mental health assistant. I'm very stressed because I
have a group project in last semester of my college and I want to get first
honours, I'm scared. Please help me cope with stress.

Model Answer:
 It's understandable that you're feeling stressed about the group project in your
last semester of college. It's a challenging time, but I can offer some
suggestions to help you cope with the stress.  1. Take slow, deep breaths: When
you're feeling overwhelmed, try taking slow, deep breaths. This can help
activate your body's relaxation response and reduce stress levels.  2.
Prioritize self-care: Make sure to take care of yourself physically and
mentally. Ensure you're getting enough sleep, eating nutritious meals, and
engaging in activities that bring you joy and relaxation.  3. Set realistic
goals: Remember that it's okay to set small, achievable goals for yourself.
Focus on what you can accomplish rather than trying to do everything at once.
4. Reach

# Inference for Human Evaluation

In [5]:
# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description.  The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. Another trigger for my anxiety is crowded places. Whenever I'm in a crowded environment, such as shopping malls or public transportation, I feel overwhelmed and suffocated. The noise, the people, and the lack of personal space make me extremely anxious. Just last week, I had to leave a store because it was too crowded, and I felt trapped and panicked. [/INST]"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=300, top_p=0.9)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question:
 <s>[INST] You are a helpful mental health counselling assistant, please answer
the mental health questions based on the patient's description.  The assistant
gives helpful, comprehensive, and appropriate answers to the user's questions.
Another trigger for my anxiety is crowded places. Whenever I'm in a crowded
environment, such as shopping malls or public transportation, I feel overwhelmed
and suffocated. The noise, the people, and the lack of personal space make me
extremely anxious. Just last week, I had to leave a store because it was too
crowded, and I felt trapped and panicked.

Model Answer:
 It can be challenging to navigate crowded places, especially when they trigger
anxiety for you. The noise, people, and lack of personal space can indeed make
such environments overwhelming and suffocating. It's important to remember that
you're not alone in feeling this way, as many people experience similar feelings
when they are in crowded places.  One suggestion I have is to t

In [6]:
# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description.  The assistant gives helpful, comprehensive, and appropriate answers to the user's questions.  I've been wrestling with the decision to visit my mother-in-law, torn between concern for her health and the disagreement with my husband about her safety. Memories of her last days weigh heavily on my mind, and I can't ignore the deterioration in her condition at 89. The stroke she suffered left her with limited strength in her left leg, and her sedentary lifestyle only worsens her health. I've been considering the best approach to approach this delicate situation. [/INST]"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=300, top_p=0.9)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question:
 <s>[INST] You are a helpful mental health counselling assistant, please answer
the mental health questions based on the patient's description.  The assistant
gives helpful, comprehensive, and appropriate answers to the user's questions.
I've been wrestling with the decision to visit my mother-in-law, torn between
concern for her health and the disagreement with my husband about her safety.
Memories of her last days weigh heavily on my mind, and I can't ignore the
deterioration in her condition at 89. The stroke she suffered left her with
limited strength in her left leg, and her sedentary lifestyle only worsens her
health. I've been considering the best approach to approach this delicate
situation.

Model Answer:
 Your situation is indeed complex, and it's understandable that you're feeling
torn between your responsibilities to your mother-in-law and your own well-
being. The emotional weight of her condition and the potential impact on her
quality of life are significant co

# Evaluation

In [ ]:
!pip install evaluate bert_score transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.3 MB/s eta 0:00:00


# Testing on 20% Test data

In [ ]:
dataset['test']

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3217
})

In [ ]:
import textwrap
import torch
import evaluate

# 20% test data for evaluation
test_data = dataset["test"]
subset_size = int(0.2 * len(test_data))

prompts = []
references = []

# Build prompts and references
for i in range(subset_size):
    sample = test_data[i]
    instruction = sample['instruction']
    input_text = sample['input']
    output_text = sample['output']

    if input_text:
        prompt = f"<s>[INST] {instruction} {input_text} [/INST]"
    else:
        prompt = f"<s>[INST] {instruction} [/INST]"

    prompts.append(prompt)
    references.append(output_text)  # reference answer

# Generate responses for each prompt and format them
generated_texts = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.7,
            use_cache=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the response
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Presentation
    try:
        question, model_answer = decoded_output.split("[/INST]", 1)
    except ValueError:
        question, model_answer = decoded_output, ""


    question = "\n".join(textwrap.wrap(question, width=80))
    model_answer = "\n".join(textwrap.wrap(model_answer, width=80))

    print("Question:\n", question)
    print("\nModel Answer:\n", model_answer)
    print("\n" + "="*80 + "\n")

    # Append decoded output to generated_texts for evaluation
    generated_texts.append(decoded_output)



!pip install rouge_score
# Initialize evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Calculate Scores
bleu_score = bleu.compute(predictions=generated_texts, references=[[ref] for ref in references])
rouge_score = rouge.compute(predictions=generated_texts, references=references)
bert_score = bertscore.compute(predictions=generated_texts, references=references, lang="en")

# Print Results
print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)
print("BERTScore (Precision, Recall, F1):", bert_score["precision"], bert_score["recall"], bert_score["f1"])


from huggingface_hub import Repository

repo = Repository(local_dir="./evaluation_results", clone_from="DiatWork/GPT-Neox-MentalHealth-Finetune-10epoch")

# Save metrics to JSON file
evaluation_metrics = {
    "ROUGE": rouge_score,
    "BERTScore": bert_score,
    "BLEU": bleu_score,
}

import json

output_file = "evaluation_metrics.json"
with open(output_file, "w") as json_file:
    json.dump(evaluation_metrics, json_file, indent=4)



Streaming output truncated to the last 5000 lines.
or when considering the potential impact of certain actions or choices.  The
first major disagreement about finances may have triggered these concerns about
the ability to resolve conflicts successfully. Anxiety, sleeplessness, and a
constant preoccupation with trying to be perfect partner can be challenging for
both you and your spouse. Obsessing over small details and becoming overly
concerned about trivial issues can heighten stress levels for both of you.  In
terms of coping strategies, open communication is a valuable aspect of any
relationship. However, it's understandable that fear of judgment or causing
further conflict may make it difficult for you to fully articulate your
emotions. Here are a few suggestions to address this issue:  1. Take the
initiative: Remind yourself that you have the right to express your concerns
without fear of judgment. Instead of waiting for your partner to bring up the
topic, take the initiative to 

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/DiatWork/GPT-Neox-MentalHealth-Finetune into local empty directory.


Download file model-00001-of-00002.safetensors:   0%|          | 8.00k/4.65G [00:00<?, ?B/s]

Download file model-00002-of-00002.safetensors:   0%|          | 32.0k/300M [00:00<?, ?B/s]

Clean file model-00002-of-00002.safetensors:   0%|          | 1.00k/300M [00:00<?, ?B/s]

Clean file model-00001-of-00002.safetensors:   0%|          | 1.00k/4.65G [00:00<?, ?B/s]

In [ ]:
import json
import numpy as np

file_path = '/content/evaluation_metrics.json'

with open(file_path, 'r') as file:
    gemma_data = json.load(file)

# Calculate Avg. BERTScore
bert_score = gemma_data["BERTScore"]

precision_avg = np.mean(bert_score["precision"])
recall_avg = np.mean(bert_score["recall"])
f1_avg = np.mean(bert_score["f1"])

precision_avg, recall_avg, f1_avg


(0.8575356042774431, 0.8734564256519775, 0.8652559844183143)